In [3]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm

In [4]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [5]:
raw_datasets["train"][0]

{'NNQT_question': 'What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}',
 'paraphrased_question': "What is Delta Air Line's periodical literature mouthpiece?",
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'sparql_dbpedia18': 'select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q188920> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P2813> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?obj . ?obj <http://www.wikidata.org/entity/P31> <http://wikidata.dbpedia.org/resource/Q1002697> } ',
 'sparql_wikidata': ' select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 } ',
 'subgraph': 'simple question right',
 'template': ' <S P ?O ; ?O instanceOf Type>',
 'template_index': 65,
 'uid': 19719}

In [56]:
rep_dict = {
    "ASK": "ask",
    "WHERE": "where",
    "SELECT": "select",
    "{": "[",
    "}": "]",
    "DISTINCT": "distinct",
    "ORDER": "order",
    "LIMIT": "limit",
    "FILTER": "filter",
    "?sbj": "?subject",
    "?obj": "?object",
}
rep_dict2 = {"{": "", "}": ""}


def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text


## Goal: Trim + Replace LowerCase + Remove weirdly long Question + replaceQ/P

In [8]:
import pandas as pd
import re
kdwd_date = '2021-10-11'
df_q = pd.read_csv(f'../../../data/kdwd/{kdwd_date}/q.csv')
df_p = pd.read_csv(f'../../../data/kdwd/{kdwd_date}/p.csv')

In [16]:
df_q.set_index('Unnamed: 0', inplace=True)
df_p.set_index('Unnamed: 0', inplace=True)

In [57]:
df_p[df_p.id==19]

,id,en
Unnamed: 0,,
7,19,place of birth
7041,19,birthplace
7042,19,born in
7043,19,pob
7044,19,birth place
7045,19,location born
7046,19,born at
7047,19,birth location
7048,19,location of birth


In [31]:
df_p[df_p.id==19].en.iloc[0].replace(" ", "_").replace("-", "_")

'place_of_birth'

In [48]:
# id to str
def encode_ids(x, df):
    try:
        return df[df.id==int(x[1:])].en.iloc[0]
    except:
        return x

In [53]:
def encode_props(qry):
    qry = replace_all(qry, rep_dict).strip()
    # Q
    for m in re.finditer(":Q\d+", qry):
        x = m.group(0)[1:]
        newstring = encode_ids(x, df_q).replace(" ", "_").replace("-", "_")
        qry = qry.replace(x, newstring)
    # P
    for m in re.finditer(":P\d+", qry):
        x = m.group(0)[1:]
        newstring = encode_ids(x, df_p).replace(" ", "_").replace("-", "_")
        qry = qry.replace(x, newstring)
    return qry
# Test
encode_props('SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),\'2009\')) }')
# "select ?obj where [ wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]"

"select ?object where [ wd:somalia p:population ?s . ?s ps:population ?object . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]"

In [54]:
encode_props("SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q58863414 . ?sbj wdt:P2541 wd:Q62900839 . ?sbj rdfs:label ?sbj_label . FILTER(CONTAINS(lcase(?sbj_label), 'model')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25")

"select distinct ?subject ?subject_label where [ ?subject wdt:instance_of wd:female_beauty_pageant . ?subject wdt:operating_area wd:all_countries . ?subject rdfs:label ?subject_label . filter(CONTAINS(lcase(?subject_label), 'model')) . filter (lang(?subject_label) = 'en') ] limit 25"

In [66]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = encode_props(d['sparql_wikidata'])
            if len(d['question'])<250: 
                q = d['question'].strip().strip("\"").lower()
                q = replace_all(q, rep_dict2)
                df = df.append({col: {'en':q, 'sparql': qry}}, ignore_index=True)
            if d['paraphrased_question']!=[] and len(d['paraphrased_question'])<250:
                q = d['paraphrased_question'].strip().strip("\"").lower()
                q = replace_all(q, rep_dict2)
                df = df.append({col: {'en':q, 'sparql': qry}}, ignore_index=True)
        except: 
            print("err")
    return df

In [67]:
df_test = prepare(raw_datasets["test"])

 57%|█████▋    | 2732/4781 [01:24<01:01, 33.22it/s]

err


 84%|████████▍ | 4032/4781 [02:04<00:23, 32.21it/s]

err


100%|██████████| 4781/4781 [02:27<00:00, 32.38it/s]


In [69]:
df_train = prepare(raw_datasets["train"])

 51%|█████     | 9792/19293 [05:00<04:21, 36.32it/s]

err


100%|██████████| 19293/19293 [10:05<00:00, 31.86it/s]


In [72]:
print(df_test.shape)
print(df_train.shape)

(9547, 1)
(38513, 1)


In [70]:
df_test.head(2)

,translation
0,{'en': 'who is the country for head of state ...
1,{'en': 'what country is mahmoud abbas the head...


In [71]:
df_all = pd.concat([df_train, df_test])
print(df_all.shape)
# shuffling all
df_all  = df_all.sample(frac = 1).reset_index(drop=True)
print(df_all.shape)

(48060, 1)
(48060, 1)


In [73]:
from sklearn.model_selection import train_test_split
df_train2, df_test2 = train_test_split(df_all, test_size=0.2)

In [76]:
print(df_test2.shape)
print(df_train2.shape)

(9612, 1)
(38448, 1)


In [77]:
ds_train = Dataset.from_pandas(df_train2)
ds_test = Dataset.from_pandas(df_test2)

In [79]:
pd.options.display.max_colwidth = 100
df_test.head()

,translation
0,"{'en': 'who is the country for head of state of mahmoud abbas', 'sparql': 'select distinct ?sub..."
1,"{'en': 'what country is mahmoud abbas the head of state of?', 'sparql': 'select distinct ?subjec..."
2,"{'en': 'what was the population of somalia in 2009-0-0?', 'sparql': 'select ?object where [ wd:s..."
3,"{'en': 'as of 2009, how many people lived in somalia?', 'sparql': 'select ?object where [ wd:som..."
4,"{'en': 'which female actress is the voice over on south park and is employed as a singer?', 'spa..."


In [80]:
df_test.iloc[0].translation    

{'en': 'who is the  country for head of state of mahmoud abbas',
 'sparql': 'select distinct ?subject where [ ?subject wdt:head_of_state wd:mahmoud_abbas . ?subject wdt:instance_of wd:country ]'}

In [82]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [83]:
ds_path='../../../data/dataset/lc-quad-wikidata-2021-10-11'
mother_ds.save_to_disk(ds_path)

In [84]:
df_train2.to_csv(f'{ds_path}/train.csv')
df_test2.to_csv(f'{ds_path}/test.csv')

In [85]:
df_test2

,translation
38355,"{'en': 'who colors of player of nico yennaris ?', 'sparql': 'select ?answer where [ wd:nico_yenn..."
17094,"{'en': 'what are the basic forms of government in syria?', 'sparql': 'select (COUNT(?object) AS ..."
41619,"{'en': 'for what was isabelle adjani nominated for in the year 1995.', 'sparql': 'select ?object..."
18625,"{'en': 'what is the origin of aiesec, that borders ans?', 'sparql': 'select ?answer where [ wd:a..."
33956,"{'en': 'how long is passover in israel?', 'sparql': 'select ?object where [ wd:passover p:durati..."
...,...
4151,"{'en': 'which is the way to determine the number of out of school children in north africa?', 's..."
33487,"{'en': 'let me know almost moon of saturn begins with the letter t', 'sparql': 'select distinct ..."
5822,"{'en': 'where is the bombarded city of nagasaki located?', 'sparql': 'select distinct ?subject w..."
34431,"{'en': 'the electric conductivity of galinstan is equivalent to 3460000?', 'sparql': 'ask where ..."
